In [34]:
from PIL import Image, ImageDraw
import imageio
import numpy as np
from io import BytesIO
import cairosvg
from lxml import etree

# Assuming WATERMARK and WATERMARK_SIZE are defined elsewhere, e.g.,
WATERMARK = """
<svg width="256" height="256" viewBox="0 0 256 256" fill="none" xmlns="http://www.w3.org/2000/svg">
  <path d="M60.0859 196.8C65.9526 179.067 71.5526 161.667 76.8859 144.6C79.1526 137.4 81.4859 129.867 83.8859 122C86.2859 114.133 88.6859 106.333 91.0859 98.6C93.4859 90.8667 95.6859 83.4 97.6859 76.2C99.8193 69 101.686 62.3333 103.286 56.2C110.619 56.2 117.553 55.8 124.086 55C130.619 54.2 137.686 53.4667 145.286 52.8C144.886 55.7333 144.419 59.0667 143.886 62.8C143.486 66.4 142.953 70.2 142.286 74.2C141.753 78.2 141.153 82.3333 140.486 86.6C139.819 90.8667 139.019 96.3333 138.086 103C137.153 109.667 135.886 118 134.286 128H136.886C140.753 117.867 143.953 109.467 146.486 102.8C149.019 96 151.086 90.4667 152.686 86.2C154.286 81.9333 155.886 77.8 157.486 73.8C159.219 69.6667 160.819 65.8 162.286 62.2C163.886 58.4667 165.353 55.2 166.686 52.4C170.019 52.1333 173.153 51.8 176.086 51.4C179.019 51 181.953 50.6 184.886 50.2C187.819 49.6667 190.753 49.2 193.686 48.8C196.753 48.2667 200.086 47.6667 203.686 47C202.353 54.7333 201.086 62.6667 199.886 70.8C198.686 78.9333 197.619 87.0667 196.686 95.2C195.753 103.2 194.819 111.133 193.886 119C193.086 126.867 192.353 134.333 191.686 141.4C190.086 157.933 188.686 174.067 187.486 189.8L152.686 196C152.686 195.333 152.753 193.533 152.886 190.6C153.153 187.667 153.419 184.067 153.686 179.8C154.086 175.533 154.553 170.8 155.086 165.6C155.753 160.4 156.353 155.2 156.886 150C157.553 144.8 158.219 139.8 158.886 135C159.553 130.067 160.219 125.867 160.886 122.4H159.086C157.219 128 155.153 133.933 152.886 140.2C150.619 146.333 148.286 152.6 145.886 159C143.619 165.4 141.353 171.667 139.086 177.8C136.819 183.933 134.819 189.8 133.086 195.4C128.419 195.533 124.419 195.733 121.086 196C117.753 196.133 113.886 196.333 109.486 196.6L115.886 122.4H112.886C112.619 124.133 111.953 127.067 110.886 131.2C109.819 135.2 108.553 139.867 107.086 145.2C105.753 150.4 104.286 155.867 102.686 161.6C101.086 167.2 99.5526 172.467 98.0859 177.4C96.7526 182.2 95.6193 186.2 94.6859 189.4C93.7526 192.467 93.2193 194.2 93.0859 194.6L60.0859 196.8Z" fill="white"/>
</svg>
"""
WATERMARK_SIZE = 40

# def add_watermark_to_image_sequence(pil_images, base_blob_name):
#     """
#     Adds a watermark to a sequence of PIL images.

#     Args:
#       pil_images: A list of PIL Image objects.
#       base_blob_name: The base name of the blob (used for naming the output).

#     Returns:
#       A list of bytes objects representing the watermarked images.
#     """

#     watermarked_images = []
#     for img in pil_images:
#         img = add_watermark_to_image(img)
        
#         # Save the image to a bytes buffer
#         buffer = BytesIO()
#         img.save(buffer, format="JPEG")
#         watermarked_images.append(buffer.getvalue())

#     return watermarked_images
  
  
def add_watermark_to_image_sequence(pil_images, base_blob_name):
    """
    Adds a watermark to a sequence of PIL images and saves as a WEBP animation.

    Args:
      pil_images: A list of PIL Image objects.
      base_blob_name: The base name of the blob (used for naming the output).

    Returns:
      The filename of the saved WEBP animation.
    """

    watermarked_images = []
    for img in pil_images:
        img = add_watermark_to_image(img)
        watermarked_images.append(img)

    # Save the images as a WEBP animation
    output_filename = f"{base_blob_name.split('.')[0]}_watermarked.webp"
    imageio.mimsave(output_filename, watermarked_images, fps=24)  # Adjust fps as needed

    return output_filename

def add_watermark_to_image(img):
    """
    Adds a watermark to a single PIL Image.

    Args:
      img: A PIL Image object.

    Returns:
      A PIL Image object with the watermark added.
    """

    # Calculate position (bottom right corner)
    padding = 12
    x = img.width - WATERMARK_SIZE - padding
    y = img.height - WATERMARK_SIZE - padding

    background_brightness = analyze_background_brightness(img, x, y, WATERMARK_SIZE)
    print(f"background_brightness: {background_brightness}")

    # Generate watermark image (replace this with your actual watermark generation)
    watermark = generate_watermark(WATERMARK_SIZE, background_brightness)

    # Overlay the watermark
    img.paste(watermark, (x, y), watermark)

    return img


def analyze_background_brightness(img, x, y, size):
    """
    Analyzes the average brightness of a region in the image.

    Args:
      img: A PIL Image object.
      x: The x-coordinate of the top-left corner of the region.
      y: The y-coordinate of the top-left corner of the region.
      size: The size of the region (square).

    Returns:
      The average brightness of the region as an integer.
    """
    region = img.crop((x, y, x + size, y + size))
    pixels = np.array(region)
    total_brightness = np.sum(
        0.299 * pixels[:, :, 0] + 0.587 * pixels[:, :, 1] + 0.114 * pixels[:, :, 2]
    ) / 1000
    print(f"total_brightness: {total_brightness}")
    return  max(0, min(255, total_brightness)) 

def generate_watermark(size, background_brightness):
    """
    Generates a watermark image from an SVG string.

    Args:
      size: The size of the watermark (square).
      background_brightness: The background brightness at the watermark position.

    Returns:
      A PIL Image object representing the watermark.
    """

    # Determine watermark color based on background brightness
    watermark_color = (0, 0, 0, 165) if background_brightness > 128 else (255, 255, 255, 165)

    # Parse the SVG string
    svg_tree = etree.fromstring(WATERMARK)

    # Find the path element and set its fill attribute
    path_element = svg_tree.find(".//{http://www.w3.org/2000/svg}path")
    if path_element is not None:
        r, g, b, a = watermark_color
        fill_color = f"rgba({r},{g},{b},{a/255})"  # Convert to rgba string
        path_element.set("fill", fill_color)

    # Convert the modified SVG tree back to a string
    modified_svg = etree.tostring(svg_tree, encoding="unicode")

    # Render the modified SVG to a PNG image with a transparent background
    png_data = cairosvg.svg2png(
        bytestring=modified_svg,
        output_width=size,
        output_height=size,
        background_color="transparent"
    )
    watermark_img = Image.open(BytesIO(png_data))

    # Convert the watermark to RGBA to handle transparency
    watermark_img = watermark_img.convert("RGBA")

    return watermark_img

In [35]:
# Load the webp image using the requests library
import time
import requests


image_url = "https://media.memedeck.xyz/memes/user:d38ee417_a500_4cd4_a455_432c3cbb61fe/video_gen/video_72b0b56c_0c40_40ff_89d2_2dd1cb01a163_2fbca50b_3e1b_42e6_9391_c2b3efa091ad.webp"  # Example webp image URL
response = requests.get(image_url, stream=True)
response.raise_for_status()  # Raise an exception for bad status codes

# Open the image using PIL
img = Image.open(response.raw)

# img_to_tensor = img_to_tensor(img)

start_time = time.time()
# Add the watermark

watermarked_img = add_watermark_to_image_sequence(img, 'video_72b0b56c_0c40_40ff_89d2_2dd1cb01a163_2fbca50b_3e1b_42e6_9391_c2b3efa091ad')
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

# Save the watermarked image (optional)
# watermarked_img.save("watermarked_image.webp")

AttributeError: 'numpy.ndarray' object has no attribute 'width'

In [27]:
!pip install lxml